In [29]:
import numpy as np
import pandas as pd



In [30]:
df = pd.read_csv("../data/Walmart.csv")

In [31]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


Fix Date and basic date features

In [32]:
# 1. Convert Date to Datetime
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

# Sort again just to be safe for lag features
df = df.sort_values(['Store', 'Date']).reset_index(drop=True)

# 2. Add Date Features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week
df['Is_Year_End'] = df['Date'].dt.is_year_end.astype(int)

# Time Index (Weeks since start of data)
start_date = df['Date'].min()
df['Time_Idx'] = ((df['Date'] - start_date).dt.days // 7).astype(int)

# Season Mapping
def get_season(month):
    if month in [12, 1, 2]: return 'Winter'
    elif month in [3, 4, 5]: return 'Spring'
    elif month in [6, 7, 8]: return 'Summer'
    else: return 'Fall'

df['Season'] = df['Month'].apply(get_season)

In [33]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Is_Year_End,Time_Idx,Season
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,0,0,Winter
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,6,0,1,Winter
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,7,0,2,Winter
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,8,0,3,Winter
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,9,0,4,Spring


Holiday distance features

In [34]:
# Get unique holiday dates
holiday_dates = df.loc[df['Holiday_Flag'] == 1, 'Date'].unique()
holiday_dates = np.sort(pd.to_datetime(holiday_dates))

# Helper function to find nearest date
def get_weeks_diff(current_date, dates, direction='forward'):
    if direction == 'forward':
        future_dates = dates[dates >= current_date]
        if len(future_dates) == 0: return 0 # or NaN
        return (future_dates[0] - current_date).days / 7
    else: # backward
        past_dates = dates[dates <= current_date]
        if len(past_dates) == 0: return 0 # or NaN
        return (current_date - past_dates[-1]).days / 7

# Apply (This might take a few seconds)
# Note: merge_asof is faster for large data, but apply is simpler for this size
df['Weeks_To_Next_Holiday'] = df['Date'].apply(lambda x: get_weeks_diff(x, holiday_dates, 'forward'))
df['Weeks_Since_Last_Holiday'] = df['Date'].apply(lambda x: get_weeks_diff(x, holiday_dates, 'backward'))

In [35]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Is_Year_End,Time_Idx,Season,Weeks_To_Next_Holiday,Weeks_Since_Last_Holiday
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,0,0,Winter,1.0,0.0
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010,2,6,0,1,Winter,0.0,0.0
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010,2,7,0,2,Winter,29.0,1.0
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010,2,8,0,3,Winter,28.0,2.0
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010,3,9,0,4,Spring,27.0,3.0


interaction and economic features

In [36]:
# Unemployment & Holiday interaction
df['Unemployment_Holiday_Combo'] = df['Unemployment'] * df['Holiday_Flag']

# Fuel Price interactions
df['Fuel_Unemployment'] = df['Fuel_Price'] * df['Unemployment']
# "Sudden change" - difference from last week
df['Fuel_Price_Diff'] = df.groupby('Store')['Fuel_Price'].diff().fillna(0)

# CPI Features
# Rolling trend over 4 weeks per store
df['CPI_Trend_4w'] = df.groupby('Store')['CPI'].diff().rolling(4).mean().reset_index(0, drop=True).fillna(0)
df['CPI_Unemployment'] = df['CPI'] * df['Unemployment']
df['CPI_Time_Idx'] = df['CPI'] * df['Time_Idx']

temperature and seasonal features

In [37]:
# Calculate Mean and Std of Temperature per Season
season_stats = df.groupby('Season')['Temperature'].agg(['mean', 'std']).to_dict()

def calculate_temp_zscore(row):
    mean = season_stats['mean'][row['Season']]
    std = season_stats['std'][row['Season']]
    return (row['Temperature'] - mean) / std

df['Temp_ZScore'] = df.apply(calculate_temp_zscore, axis=1)

# Flags for abnormal temps
df['Abnormally_Hot'] = (df['Temp_ZScore'] > 2).astype(int)
df['Abnormally_Cold'] = (df['Temp_ZScore'] < -2).astype(int)

lag features (for target variable:weekly sales)

In [38]:
# Lag 1 week sales
df['Lag_Sales_1w'] = df.groupby('Store')['Weekly_Sales'].shift(1)

# Average of last 4 weeks (excluding current, hence shift(1) first)
df['Rolling_Mean_Sales_4w'] = df.groupby('Store')['Weekly_Sales'].shift(1).rolling(window=4).mean().reset_index(0, drop=True)

# Drop NaN values created by lags (first 4 weeks will be empty)
df = df.dropna().reset_index(drop=True)

In [39]:
# Define split dates for expanding window validation
# Example splits based on your text:
split_dates = [
    pd.Timestamp('2012-02-01'), # Train: ~2010-2011, Test: Early 2012
    pd.Timestamp('2012-06-01'), # Train: ~2010-Mid 2012, Test: Mid-Late 2012
]

for split_date in split_dates:
    print(f"--- Splitting at {split_date} ---")
    
    # 1. Split Data
    train_data = df[df['Date'] < split_date].copy()
    test_data = df[df['Date'] >= split_date].copy()
    
    # Stop if we ran out of test data (optional check)
    if len(test_data) == 0: break
    
    # 2. Target Encoding (Store Mean) - CALCULATED ON TRAIN ONLY
    store_means = train_data.groupby('Store')['Weekly_Sales'].mean().to_dict()
    
    # Apply to Train
    train_data['Store_Target_Mean'] = train_data['Store'].map(store_means)
    
    # Apply to Test (using values learned from Train)
    test_data['Store_Target_Mean'] = test_data['Store'].map(store_means)
    
    # Handle stores in test that weren't in train (unlikely for Walmart data, but good practice)
    global_mean = train_data['Weekly_Sales'].mean()
    test_data['Store_Target_Mean'] = test_data['Store_Target_Mean'].fillna(global_mean)

    # ... Now you can run your model on train_data and evaluate on test_data ...
    print(f"Train Shape: {train_data.shape}, Test Shape: {test_data.shape}")
    # RunModel(train_data, test_data) 

--- Splitting at 2012-02-01 00:00:00 ---
Train Shape: (4500, 28), Test Shape: (1755, 28)
--- Splitting at 2012-06-01 00:00:00 ---
Train Shape: (5265, 28), Test Shape: (990, 28)
